In [86]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D,BatchNormalization,Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [87]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [88]:
img_gen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rescale=1./255)

In [89]:
train_gen = img_gen.flow_from_directory('../../../data_set/模型訓練資料/format_train/',target_size=(96,96),batch_size=32)
val_gen = img_gen.flow_from_directory('../../../data_set/模型訓練資料/format_val/',target_size=(96,96),batch_size=32)

Found 55167 images belonging to 800 classes.
Found 13637 images belonging to 800 classes.


In [90]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus),
                  "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)


solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


In [91]:
input_shape = (96,96,3)

convnet = Sequential()
convnet.add(
    Conv2D(64, (10, 10),
           activation='relu',
           input_shape=input_shape,
           kernel_initializer=initializers.RandomNormal(stddev=0.01),
           kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128, (7, 7), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256, (7, 7), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(512, (4, 4), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(1024, (2, 2), activation='relu',padding='same',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(Dense(1024, activation="relu", kernel_regularizer=l2(1e-3)))
convnet.add(Dropout(0.51))
convnet.add(Dense(800, activation="softmax"))

convnet.compile(optimizer= Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
convnet.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 87, 87, 64)        19264     
_________________________________________________________________
batch_normalization_23 (Batc (None, 87, 87, 64)        256       
_________________________________________________________________
dropout_39 (Dropout)         (None, 87, 87, 64)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 37, 37, 128)       401536    
_________________________________________________________________
dropout_40 (Dropout)         (None, 37, 37, 128)       0         
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 18, 18, 128)      

In [ ]:
convnet.fit(train_gen,steps_per_epoch=train_gen.samples/32,validation_data=val_gen,
           epochs=20,verbose=True)

Epoch 1/20
1723/1723 [==============================] - 391s 227ms/step - loss: 4.1275 - accuracy: 0.2369 - val_loss: 7.4700 - val_accuracy: 0.0337
Epoch 2/20
1723/1723 [==============================] - 691s 401ms/step - loss: 4.0218 - accuracy: 0.2563 - val_loss: 5.6084 - val_accuracy: 0.1184
Epoch 3/20
1723/1723 [==============================] - 677s 393ms/step - loss: 3.9451 - accuracy: 0.2707 - val_loss: 6.0228 - val_accuracy: 0.0825
Epoch 4/20
1723/1723 [==============================] - 718s 417ms/step - loss: 3.8724 - accuracy: 0.2882 - val_loss: 5.6197 - val_accuracy: 0.1317
Epoch 5/20
1723/1723 [==============================] - 742s 431ms/step - loss: 3.8082 - accuracy: 0.2992 - val_loss: 5.7011 - val_accuracy: 0.1098
Epoch 6/20
1193/1723 [===================>..........] - ETA: 3:36 - loss: 3.7208 - accuracy: 0.3118

In [16]:
convnet.save('../../../data_set/模型訓練資料/save/original_cnn.h5')